In [1]:
import pandas as pd
import requests
from selenium import webdriver
from bs4 import BeautifulSoup

In [15]:
categories = {
    100: "정치",
    101: "경제",
    102: "사회",
    103: "생활/문화",
    104: "세계",
    105: "IT/과학",
}

In [3]:
category, date, page = 100, 20191121, 1

url = "https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1={}&date={}&page={}".format(category, date, page)

In [4]:
driver = webdriver.Chrome()
driver.get(url)

In [5]:
articles = driver.find_elements_by_css_selector(".list_body > ul > li")
len(articles)

20

In [6]:
title = articles[0].find_element_by_css_selector("dt:not(.photo) > a").text
link = articles[0].find_element_by_css_selector("dt:not(.photo) > a").get_attribute("href")
title, link

('北, 文대통령 초청 친서에 "김 위원장 참석이유 못찾아, 이해바란다"',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=123&aid=0002215174')

In [7]:
response = requests.get(link)
dom = BeautifulSoup(response.content, "html.parser")

In [10]:
content = dom.select_one("#articleBodyContents").text.replace("\n", "").replace("\'", "").split("{}")[1].strip()
content

'"이 기회에 북남관계 풀기 위한 문 대통령의 고뇌와 번민 이해""과연 지금이 북남수뇌분 만날 때인가" 비판과 함께 초청 거부 밝혀◆…북한은 내주 부산 한-아세안 정상회의에 문재인 대통령이 김정은 국무위원장을 초청한 데 대해 "참석해야 할 이유를 찾지 못했다"고 공식 거부 입장을 밝혔다. 사진은 지난 6월 30일 판문점 남북미 정상간 극적 만남의 모습(청와대)김정은 북한 국무위원장이 오는 25일부터 부산에서 열리는 한·아세안 특별정상회의에 참석해달라는 문재인 대통령의 초청을 공식 거부했다.<조선중앙통신>은 21일 "지난 11월 5일 남조선의 문재인 대통령은 조선민주주의인민공화국 국무위원회 위원장께서 이번 특별수뇌자회의에 참석해주실 것을 간절히 초청하는 친서를 정중히 보내왔다"며 문 대통령이 초청 친서를 보낸 사실을 공개했다.통신은 그러면서 "친서가 국무위원회 위원장에 대한 진정으로 되는 신뢰심과 곡진한 기대가 담긴 초청이라면 굳이 고맙게 생각하지 않을 까닭이 없다"면서도 "남측의 기대와 성의는 고맙지만 국무위원회 위원장께서 부산에 나가셔야 할 합당한 리유(이유)를 끝끝내 찾아내지 못한 데 대해 리해(이해)해주길 바란다"며 불참을 통고했다.\u3000\u3000그동안 한-아세안 특별정상회의에 김 위원장을 초대할 것인 가에 대해 묵묵부답을 해 왔던 청와대로서는 북측의 이번 발표로 초대장을 보낸 게 공식적으로 확인 된 셈이라 다소 당황스러운 모양새다.매체는 또 "우리는 보내온 친서가 국무위원회 위원장에 대한 진정으로 되는 신뢰심과 곡진한 기대가 담긴 초청이라면 굳이 고맙게 생각하지 않을 까닭이 없다"면서도 "하지만 흐려질대로 흐려진 남조선의 공기는 북남관계에 대해 매우 회의적이며 남조선당국도 북남사이에 제기되는 모든 문제를 의연히 민족공조가 아닌 외세의존으로 풀어나가려는 그릇된 립장에서 탈피하지 못하고 있는 것이 오늘의 엄연한 현실"이라고 꼬집었다.이어 "우리는 남측이 국무위원회 위원장의 부산방문과 관련한 경호와 의전 등 모든 영접준비를 최상의 수준에서 갖추어놓고 

### 함수로 만들기

In [13]:
def get_articles(category, date, page):
    url = "https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1={}&date={}&page={}".format(category, date, page)
    driver = webdriver.Chrome()
    driver.get(url)
    articles = driver.find_elements_by_css_selector(".list_body > ul > li")
    datas = []
    for article in articles:
        title = article.find_element_by_css_selector("dt:not(.photo) > a").text
        link = article.find_element_by_css_selector("dt:not(.photo) > a").get_attribute("href")
        response = requests.get(link)
        dom = BeautifulSoup(response.content, "html.parser")
        content = dom.select_one("#articleBodyContents").text.replace("\n", "").replace("\'", "").split("{}")[1].strip()
        datas.append({
            "title": title,
            "link": link,
            "content": content,
            "category": category,
        })
    driver.quit()
        
    return pd.DataFrame(datas)

In [14]:
get_articles(100, 20191121, 1)

,category,content,link,title
0,100,[속보] 한국당 “현역의원 3분의 1 이상 공천 컷오프 추진할 것”박민지 기자 pm...,https://news.naver.com/main/read.nhn?mode=LSD&...,[속보] 한국당 “현역의원 3분의 1 이상 공천 컷오프 추진할 것”
1,100,[서울경제] 한국당 “현역의원 3분의 1 이상 공천 컷오프 추진”한국당 “현역의원 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,(속보) 한국당 “현역의원 3분의 1 이상 공천 컷오프 추진”
2,100,[김민우 기자 minsicht@chosunbiz.com]- Copyrights ⓒ ...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[속보]한국당 ""현역 의원 3분의 1이상 공천 컷오프"""
3,100,(서울=연합뉴스) 안정원 기자 = 21일 서울 여의도 국회에서 열린 외교통상위원회...,https://news.naver.com/main/read.nhn?mode=LSD&...,분주한 외교부
4,100,21일 서울 용산구 국방컨벤션에서 열린 2019 청렴 방위사업 세미나에서 한명진 방...,https://news.naver.com/main/read.nhn?mode=LSD&...,2019 청렴 방위사업 세미나
5,100,(부산=연합뉴스) 김주형 기자 = 이낙연 국무총리가 21일 오후 부산 해운대 벡스코...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[한-아세안] 이낙연 총리, '2019 국가 대테러종합훈련' 격려사"
6,100,(서울=연합뉴스) 신준희 기자 = 강기정 청와대 정무수석이 21일 오후 청와대 앞 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,다시 황교안 찾은 강기정
7,100,21일 서울 용산구 국방컨벤션에서 열린 2019 청렴 방위사업 세미나에서 참석자들이...,https://news.naver.com/main/read.nhn?mode=LSD&...,'청렴 다짐 합니다'
8,100,(서울=연합뉴스) 신준희 기자 = 강기정 청와대 정무수석(왼쪽)이 21일 오후 청와...,https://news.naver.com/main/read.nhn?mode=LSD&...,다시 황교안 찾은 강기정
9,100,[디지털뉴스국]▶매경이 에어팟프로 쏩니다! M코인▶네이버 메인에서 매일경제를 받아보...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[속보] 한국당 ""현역의원 3분의 1 이상 공천 컷오프 추진"""


### for문을 이용해서 데이터 수집

In [18]:
dfs = []
for category in categories:
    print(categories[category],end=" ")
    for page in range(1, 6):
        print(page, end=" ")
        df = get_articles(category, date, page)
        dfs.append(df)
    print()
articles_df = pd.concat(dfs, ignore_index=True)
articles_df.tail()

정치 1 2 3 4 5 
경제 1 2 3 4 5 
사회 1 2 3 4 5 
생활/문화 1 2 3 4 5 
세계 1 2 3 4 5 
IT/과학 1 2 3 4 5 


,category,content,link,title
595,105,엔지니어들이 SOC실증연구센터에서 스마트 건설 방식으로 공사를 진행하고 있다. 제...,https://news.naver.com/main/read.nhn?mode=LSD&...,"SKT, ‘5G+건설·기계’ 드론 띄워 측량, 굴삭기로 정밀공사 ‘착착’"
596,105,"SKT, 건기연·현대건설기계 등과 스마트 건설 실증 “스마트 건설방식을 도입하면 기...",https://news.naver.com/main/read.nhn?mode=LSD&...,드론으로 측량하고 굴삭기 정밀 작업까지 ‘척척’
597,105,"문 대통령 AI 강국 외치고, 네이버⋅소프트뱅크 세계 시장 겨냥한 AI기업 출범 합...",https://news.naver.com/main/read.nhn?mode=LSD&...,"[비즈톡톡] ""튀면 죽는다""…앞뒤 다른 정부에 갈피 못잡는 IT 기업들"
598,105,민주노총 전국공공운수노동조합(이하 공공운수노조)이 21일 세종시 어진동 과학기술정보...,https://news.naver.com/main/read.nhn?mode=LSD&...,"공공연구노조 ""출연연 용역근로자 정규직 전환 촉구"" 결의대회"
599,105,"3국 IT국장회의…공개 SW, 저작권자 허락 라이선스 따라 복제·수정·재배포 가능(...",https://news.naver.com/main/read.nhn?mode=LSD&...,"한·중·일, 5G·AI 기술 구현 위해 '공개 SW' 적극 활용 합의"


### csv파일로 저장

In [19]:
articles_df.to_csv("article.csv", index=False, encoding="utf-8-sig")